In [6]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [7]:
working_dir = os.getcwd()
windowsize = 1

In [8]:
def read_training_data():
    df = pd.read_csv(f"{working_dir}/../data/TrainData.csv", sep=',')
    windspeed_training = df["WS10"].to_numpy().reshape((-1, 1))
    power_training = df["POWER"].to_numpy()
    u = df["U10"].to_numpy()
    v = df["V10"].to_numpy()
    wind_direction_training = np.mod(180 + np.rad2deg(np.arctan2(v, u)), 360)

    return windspeed_training, power_training, wind_direction_training


def read_input_data():
    df = pd.read_csv(f"{working_dir}/../data/WeatherForecastInput.csv", sep=',')
    windspeed_input = df["WS10"].to_numpy().reshape((-1, 1))
    u_input = df["U10"].to_numpy()
    v_input = df["V10"].to_numpy()
    direction_input = np.mod(180 + np.rad2deg(np.arctan2(v_input, u_input)), 360).reshape((-1, 1))
    windspeed_and_direction_input = np.concatenate((windspeed_input, direction_input), axis=1)

    return windspeed_input, direction_input, windspeed_and_direction_input

def read_solution():
    df = pd.read_csv(f"{working_dir}/../data/Solution.csv", sep=',')
    actual_power = df["POWER"].to_numpy()
    timestamp = df["TIMESTAMP"].to_numpy()

    return actual_power, timestamp

In [9]:
def create_timeseries(powers, windowsize):
    rows = len(powers) - windowsize
    x = np.zeros((rows, windowsize))
    y = np.zeros(rows)

    for i in range(rows):
        for j in range(windowsize):
            x[i][j] = powers[i + j]

        y[i] = powers[i + windowsize]

    return x, y

In [10]:
training_data = read_training_data()
windspeed_training = training_data[0]
power_training = training_data[1]

input_data = read_input_data()
windspeed_input = input_data[0]
wind_direction = input_data[1]
windspeed_and_direction_input = input_data[2]

solution = read_solution()
actual_power = solution[0]
timestamp = solution[1]

In [11]:
x, y = create_timeseries(power_training, windowsize)
forecast_input = create_timeseries(actual_power, windowsize)[0]

In [ ]:
from sklearn.linear_model import LinearRegression


def timeseries_linear_regression(x, y, forecast_input, actual_power, windowsize):
    model = LinearRegression()
    model.fit(x, y)
    predicted_power = model.predict(forecast_input)

    actual_power_comparison = actual_power[windowsize:]

    # rmse = RMSE(actual_power_comparison, predicted_power)

    return predicted_power#, rmse

In [ ]:
ts_lin_reg = timeseries_linear_regression(x, y, forecast_input, actual_power, windowsize)

In [ ]:
# Plot the time-series figure for Linear Regression and SVR
plt.figure(figsize=(14, 6))
plt.plot(actual_power[windowsize:], label="True wind energy")
plt.plot(ts_lin_reg, label="Linear Regression")
# plt.plot(X_true_datetime, y_pred_svr, label="SVR")
plt.xlabel("Time")
plt.ylabel("Wind power")
plt.title("Wind power forecasts (Linear Regression and SVR)")
plt.legend()
plt.show()